In [3]:
MODEL = "llama2"

In [2]:

from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()
model.invoke("joke tell me")

"\nSure, here's a quick and dirty joke:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!"

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("ml.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='Machine Learning For Absolute\nBeginners\n \n \n \n \nOliver Theobald', metadata={'source': 'ml.pdf', 'page': 1}),
 Document(page_content='Second Edition\nCopyright © 2017 by Oliver Theobald\nAll rights reserved. No part of this publication may be reproduced,\ndistributed, or transmitted in any form or by any means, including\nphotocopying, recording, or other electronic or mechanical\nmethods, without the prior written permission of the publisher,\nexcept in the case of brief quotations embodied in critical reviews\nand certain other non-commercial uses permitted by copyright law.', metadata={'source': 'ml.pdf', 'page': 2}),
 Document(page_content='Contents\n \nINTRODUCTION\nWHAT IS MACHINE LEARNING?\nML CATEGORIES\nTHE ML TOOLBOX\nDATA SCRUBBING\nSETTING UP YOUR DATA\nREGRESSION ANALYSIS\nCLUSTERING\nBIAS & VARIANCE\nARTIFICIAL NEURAL NETWORKS\nDECISION TREES\nENSEMBLE MODELING\nBUILDING A MODEL IN PYTHON\nMODEL OPTIMIZATION\nFURTHER RESOURCES\nDOWNLOADING DAT

PROMPT TEMPLATE

In [5]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



In [6]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = prompt | model | parser

In [7]:
chain.invoke(
    {
        "context": "name i am given was rima",
        "question": "What is my name"
    }
)

"Hi Rima! Your name is... I'm afraid I don't know. I'm just an AI and I don't have access to personal information about individuals unless it's been explicitly provided. Can you please tell me your name again?"

In [8]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [9]:
# This will create a vector storage memory
from langchain_community.vectorstores import DocArrayInMemorySearch

vector_store = DocArrayInMemorySearch.from_documents(pages, embedding = embeddings)

In [13]:
# making a data retriever from the vector store
# Retriever is a component of Langchain that is used to retrieve the most relevant data from the data source
retriever = vector_store.as_retriever(search_kwargs={"k": 7})

retriever.invoke("What is machine learning?")

[Document(page_content='actions.\nWhile this sounds simple enough, implementation is a much more difficult\ntask and beyond the scope of an absolute beginner’s introduction to machine\nlearning. Reinforcement learning algorithms aren’t covered in this book,\nhowever, I will leave you with a link to a more comprehensive explanation of\nreinforcement learning and Q-learning following the Pac-Man scenario.\nhttps://inst.eecs.berkeley.edu/~cs188/sp12/projects/reinforcement/reinforcement.html', metadata={'source': 'ml.pdf', 'page': 20}),
 Document(page_content='# The final two columns of the table are not part of the original dataset and have been added for convenience to complete the following equation.\nWhere:\nΣ = Total sum\nΣx = Total sum of all x values (1 + 2 + 1 + 4 + 3 = 11)\nΣy = Total sum of all y values (3 + 4 + 2 + 7 + 5 = 21)\nΣxy = Total sum of x*y for each row (3 + 8 + 2 + 28 + 15 = 56)\nΣx\n2 \n= Total sum of x*x for each row (1 + 4 + 1 + 16 + 9 = 31)\nn = Total number of ro

In [16]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    } | prompt | model | parser
)

chain.invoke({"question": "What is deep learning?"})

' Based on the provided documents, deep learning refers to a subfield of machine learning that involves the use of artificial neural networks to model and solve complex problems. The term "deep" refers to the number of layers in the neural network, which can range from a few to hundreds or even thousands. Deep learning algorithms are capable of learning and representing complex patterns in data, such as images, speech, and text, and have been successful in a wide range of applications, including image recognition, natural language processing, and autonomous driving.\n\nThe documents provide some background information on machine learning and reinforcement learning, but do not go into detail on deep learning specifically. However, they do mention that reinforcement learning algorithms are not covered in the book, which suggests that deep reinforcement learning is a topic beyond the scope of the book.'

In [23]:
questions = [
    "What is machine learning?",
    "What is Supervised Learning?",
    "What is Unsupervised Learning?",
    "What is Reinforcement Learning?",
    "What is Deep Learning?",
    "What is Neural Networks?",
    "What is Convolutional Neural Networks?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is machine learning?
Answer: Machine learning is a subfield of artificial intelligence (AI) that involves developing algorithms and statistical models that enable computers to learn from data, make decisions, and improve their performance on a specific task over time. In other words, machine learning is a technique that enables computers to learn and improve their performance on a particular task without being explicitly programmed for each task.

Machine learning algorithms can be broadly classified into two types: supervised and unsupervised learning. Supervised learning involves training an algorithm on labeled data, where the output is known, and the algorithm learns to map inputs to outputs based on the patterns in the labeled data. Examples of supervised learning tasks include image classification, speech recognition, and sentiment analysis.

Unsupervised learning involves training an algorithm on unlabeled data, where there are no known outputs. The algorithm lear

In [28]:
# Best way works like answer generation of gpt
for s in chain.stream({"question": "What are bias and weights?"}):
    print(s, end="", flush=True)

Based on the provided context, bias and weights seem to be related to machine learning concepts. Here is what I know about them based on the given documents:

* Bias: In machine learning, bias refers to the gap between predicted values and actual values. It measures the error caused by an algorithm's assumptions or simplifications in representing a complex problem. Understanding bias is crucial for navigating underfitting and overfitting (Document(page_content='BIAS & VARIANCE', metadata={'source': 'ml.pdf', 'page': 51})).
* Weights: In machine learning, weights refer to the algorithm's settings or lines of code that control various aspects of a model's behavior (Document(page_content='metrics\nAlgorithm selection is an important step in forming an accurate prediction\nmodel, but deploying an algorithm with a high rate of accuracy can be a\ndifficult balancing act. The fact that each algorithm can produce vastly\ndifferent models based on the hyperparameters provided can lead to\ndrama